# 0. 배경 지식

* 형태소 (Morpheme)
  - 의미의 최소 단위. (자립, 의존, 실질, 형식 형태소로 구분)
  - e.g., 철수가 이야기책을 읽었다. -> 철수/가/ 이야기/책/을/ 읽/었/다.
  - 형태소는 2가지 종류의 Ambiguity가 있다.
      - 칸을 나누는거에 따라서 (segmenting을 어떻게 할 것인가)
      - 각각의 칸이 가지는 pos에 따라서 (pos ambiguity를 어떻게 해결할 것인가)

* 형태소 분석기를 만드는데 필요한 3가지
  - Lexicon (or Corpus or 사전)
     - entry와 POS가 있어야 한다.
  - Morphotactics (연결할 수 있나) 
     - 형태소 결합 규칙이 있어야 한다. (e.g. 명사+어미(x), 형용사+어미(o) )
  - Orthographic rules (연결 시 변화되는 모양들을 인식: 원형인식 필요 ex.stemming)
     - 불규칙 사전을 구축한다.
* 한국어 형태론적 특성
  - 단어 형성 규칙(word-formation rule) 
     - 불규칙 현상 (e.g. ㅅ불규칙(잇+어), ㄷ불규칙(걷), ㄹ불규칙(아름답), ...)
     - 음운현상 (e.g. 으탈락(쓰+어), ㄹ탈락(울), 축약(하+어 -> 해))
     - 띄어쓰기 문제 발생 (e.g. 대한 중학교 vs 대한중학교)
  - 형태론적 모호성이 있는 단어가 존재 (e.g. 이 -> 명사, 대명사, 수사, 조사, 서술격조사, 등 다된다.)

# 1. 엔트리/기능어 사전 구축

* 사전에 구축된 지식 베이스를 사용 
 - 일반어(ex.체언,용언) 단어들의 ENTRY, 기능어(ex.조사,어미)단어들의 ENTRY, 
 - 각각의 ENTRY마다 POS, INFLECTION, ANALYSIS
 - Inflection은 불규칙 사전을 구축하기 위해 필요한 정보이다.

In [1]:
from hangul_utils import split_syllables, join_jamos
import xlrd

# li_initial_phonemes = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# li_medial_phonemes = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# li_final_phonemes = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'null']
# sample_text = '가까웠지만'
# jamo = split_syllables(sample_text)
# print(jamo)
# restored_text = join_jamos(jamo)
# print(restored_text)

class Trie(list):
    
    def __init__(self):
        chs = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', 'ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', 'ㄲ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅄ', 'ㅆ']
        self.extend([ [None, None] for a in range(len(chs)) ]) # 왼쪽 None: 현재 ch의 info저장, 오른쪽 None: 다음 ch와 연결되는 노드
        self.dic = {}
        for i, ch in enumerate(chs):
            self.dic[ch] = i # look up table for 한글
        
    def index(self,ch):
        return self.dic[ch]
    
    def insert(self, word, info_list):
        word = split_syllables(word)
        if(len(word)==1):
            self.set_info(word, info_list)
        else:
            f_word = word[0]
            word = word[1:]
            char_idx = self.index(f_word)
            
            # [1]인 이유: 다음 ch와 연결되는 노드는 오른쪽 None에 저장한다.
            if (self[char_idx][1] == None):
                self[char_idx][1] = Trie()
                self[char_idx][1].insert(word,info_list)
            else:
                self[char_idx][1].insert(word,info_list)
            
    def set_info(self, f_word, _info):
        # [0]인 이유: info를 저장할 땐 왼쪽 None에 저장한다.
        if self[self.index(f_word)][0] == None:
            self[self.index(f_word)][0] = [_info] # 1개라도 배열로 처리
        else: # 중복처리
            #print(self[self.index(f_word)][0])
            temp = []
            temp += self[self.index(f_word)][0]
            temp += [_info]
            self[self.index(f_word)][0] = temp[:]
    
    def search(self, trie_list, word):
        word = split_syllables(word)
        if(len(word)==1):
            return trie_list[self.index(word)][0] # info를 출력할 땐, 왼쪽 None을 출력한다.
        else:
            f_word = word[0]
            word = word[1:]
            char_idx = self.index(f_word) 
            return self.search(trie_list[char_idx][1], word) # Search를 찾아나설 땐, 오른쪽 None에 저장된 연결된 노드들을 따라 나선다.

def search(trie, word):
    try:
        if trie.search(trie, word) == None:
            return "No such word in Trie"
        else:
            return trie.search(trie, word)
    except(TypeError):
        return "No such word in Trie"

# Trie 생성
trie = Trie()
trie_fun = Trie()
print('>> Construct Trie Complete !!')
workbook = xlrd.open_workbook('resources/voca_set.xlsx')
print('>> Load Excels Complete !!')
print('>> Excel Sheet Names: ',workbook.sheet_names())

>> Construct Trie Complete !!
>> Load Excels Complete !!
>> Excel Sheet Names:  ['엔트리사전', '기능어사전']


In [2]:
# 수정해야 되는 불규칙 정보. (예: VERB-REG1~3, ADJ-REG1~3)
# 원래대로라면, trie class에 modify함수가 있어서 수정해야함.
def infl_modify_REG(word, pos, inf):
    reg1_list = ['ㅑ','ㅒ','ㅖ','ㅘ','ㅛ','ㅙ','ㅚ','ㅝ','ㅠ','ㅡ','ㅢ','ㅣ']
    reg2_list = ['ㅗ','ㅜ']
    reg3_list = ['ㅏ','ㅓ','ㅕ','ㅐ','ㅔ','ㅞ']
    
    if inf == 'VERB-REG' or inf == 'ADJ-REG':
        word = split_syllables(word)
        list_word = list(word)
        
        if list_word[-1] in reg1_list:
            word = "".join(list_word)
            word = join_jamos(word)
            inf = inf + '1'
        elif list_word[-1] in reg2_list:
            word = "".join(list_word)
            word = join_jamos(word)
            inf = inf + '2'            
        elif list_word[-1] in reg3_list:
            word = "".join(list_word)
            word = join_jamos(word)
            inf = inf + '3'              
        else:
            word = "".join(list_word)
            word = join_jamos(word)            
            return word, pos, inf # -REG0이므로, 그냥 패스
        
        return word, pos, inf # 수정된 내용이 반영된 정보 return
    
    else:
        return word, pos, inf # -REG가 아니므로 (modify할 필요x), 그냥 패스

In [3]:
########################
""" 엔트리 사전 Corpus """
########################
worksheet1 = workbook.sheet_by_name('엔트리사전')
num_rows = worksheet1.nrows #줄 수 가져오기
num_cols = worksheet1.ncols #칸 수 가져오기

# row_val = worksheet_name.row_values(row_index) #줄 값 가져오기(list형식)
# cell_val = worksheet_name.cell_value(row_index,cell_index)

for i in range(1, num_rows):
    word = worksheet1.cell_value(i, 0)
    pos = worksheet1.cell_value(i, 1)
    inf = worksheet1.cell_value(i, 2)
    word, pos, inf = infl_modify_REG(word, pos, inf)
    
    anly = word+'/'+pos
    _info = pos+' '+inf+' '+anly
    trie.insert(word, _info) # word는 insert함수에 의해 decomposition된다.
print('>> Store 엔트리-사전 Corpus to Trie, Complete !!')   

########################
""" 기능어 사전 Corpus """
########################
worksheet2 = workbook.sheet_by_name('기능어사전')
num_rows = worksheet2.nrows #줄 수 가져오기
num_cols = worksheet2.ncols #칸 수 가져오기

# row_val = worksheet_name.row_values(row_index) #줄 값 가져오기(list형식)
# cell_val = worksheet_name.cell_value(row_index,cell_index)

for i in range(1, num_rows):
    word = worksheet2.cell_value(i, 0)
    pos = worksheet2.cell_value(i, 1)
    inf = worksheet2.cell_value(i, 2)
    anly = worksheet2.cell_value(i, 3)
    _info = pos+' '+inf+' '+anly
    decomposed_w = split_syllables(word)
    reversed_w = decomposed_w[::-1] # reverse하기 위해 decomposition 미리 실시
    trie_fun.insert(reversed_w, _info) # 이미 decomposition되어 있지만 상관없음
print('>> Store 기능어-사전 Corpus to Trie, Complete !!')    

###############################################################
# ### 사전 검색 예시
# print(search(trie, '가'))
# print(search(trie, '장'))
# print(search(trie, '잠재우'))
# print(search(trie, '가지'))
# print(search(trie, '중'))

# ### 기능어 사전 검색 예시
# ### 기능어들은 검색하기 전 미리 알아서 reverse해준다.
# print(search(trie, split_syllables('거든요')[::-1]))
# print(search(trie, split_syllables('거든')[::-1]))
# print(search(trie, split_syllables('거든하하하')[::-1]))
# print(search(trie, split_syllables('ㅆ던가요')[::-1]))

>> Store 엔트리-사전 Corpus to Trie, Complete !!
>> Store 기능어-사전 Corpus to Trie, Complete !!


In [4]:
print(search(trie, '가다듬'))

['VV VERB-REG 가다듬/VV']


# 2. 확장 사전 구축
  - 확장 사전은 크게 용언과 체언으로 나뉜다. 용언의 경우 형태소 복원 규칙에 따라서 사전을 확장한다. 체언은 형태의 변화가 없기 때문에 원래 엔트리만 등록한다.
      - 용언은 문맥에 따라 많이 바뀌기 때문에 원형을 찾아 하나로 통일하는 작업이 꼭 필요하다. (ex. stemming)
  - 불규칙 정보를 처리하기 위해 알고리즘(로직)으로 처리하는 것보다는 확장 사전을 구축한다. (모든 가능한 변형태를 확장 사전에 등록.) 그리고 하나하나 일일이 사전에 등록하기 보다는 공통되는 부분을 활용해서 사전에 등록하는 것이 필요.
  - 정보 검색에서 형태소 분석이 매우 중요하다. Retrieval되는 단위가 보통 단어이기 때문에, 문장에서 형태소 분석을 통해 단어를 어떻게 segmentation할 것인가가 중요하다. 따라서, 정보 검색을 위한 형태소 분석은 치밀하게 짜여져 있어야 한다. 즉, 형태소 분석의 정밀도(=확장 사전 정밀도)는 어플리케이션에 따라 달라질 수 있다.

In [5]:
DEBUG = False
def LOG(s):
    if DEBUG:
        print(s)

### Reload Excel File for 확장 사전 구축
# import xlrd
workbook = xlrd.open_workbook('resources/voca_set.xlsx') 
worksheet1 = workbook.sheet_by_name('엔트리사전')
num_rows = worksheet1.nrows #줄 수 가져오기
num_cols = worksheet1.ncols #칸 수 가져오기

neg_vowels = ['ㅣ','ㅡ', 'ㅜ','ㅓ', 'ㅠ', 'ㅕ', 'ㅐ'] # 음성 모음
pos_vowels = ['ㅗ', 'ㅏ','ㅛ', 'ㅑ'] # 양성 모음

for i in range(1, num_rows):  
    org_word = worksheet1.cell_value(i, 0)
    pos = worksheet1.cell_value(i, 1) # pos는 그대로 사용.
    org_inf = worksheet1.cell_value(i, 2)
    org_anly = org_word+'/'+pos
    org__info = pos+' '+org_inf+' '+org_anly    
    
    
    # -P 불규칙
    if org_inf == 'VERB-P' or org_inf == 'ADJ-P': # e.g. 눕
        LOG(org__info) # 확인
        ### VERB-P1 # e.g. 누우
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word[-1] = '우' # 'ㅂ'을 댄신해서 '우'를 삽입한다.
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        ### VERB-P2 # e.g. 누워
        if org_word == '돕' or org_word == '곱': # 예외: 돕 ---VERB-P2---> 도와
            # word
            word = split_syllables(org_word)
            list_word = list(word)
            list_word[-1] = '와'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '2'
            # anly
            anly = word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)            
        else:
            # word
            word = split_syllables(org_word)
            list_word = list(word)
            list_word[-1] = '워'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '2'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)        
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')
 

    if org_inf == 'VERB-T': # e.g. 일컫
        LOG(org__info) # 확인
        ### VERB-T1 # e.g. 일컬
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word[-1] = 'ㄹ'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')


    # -L 불규칙
    if org_inf == 'VERB-L' or org_inf == 'ADJ-L': # e.g. 이끌
        LOG(org__info)
        ### VERB-L1 # e.g. 이끄
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        del list_word[-1]
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')

        
    # -YE 불규칙
    if org_inf == 'VERB-YE' or org_inf == 'ADJ-YE': # e.g. 난파하
        LOG(org__info)
        ### VERB-YE1 # e.g. 난파해
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word[-1] = 'ㅐ'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        ### VERB-YE2 # e.g. 난파하여
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word[-1] = 'ㅏ여'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '2'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')


    # -S 불규칙
    if org_inf == 'VERB-S' or org_inf == 'ADJ-S': # e.g. 규정짓
        LOG(org__info)
        ### VERB-S1 # e.g. 규정지
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        del list_word[-1]
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')

    
    # -LU 불규칙
    if org_inf == 'VERB-LU' or org_inf == 'ADJ-LU': # (르불규칙) e.g. 흐르(음성모음), 가르(양성모음) 
        LOG(org__info)
        ### VERB-LU1 # e.g. 흘러(음성모음), 갈라(양성모음)
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        # 일단 맨뒤에 있는 '르' 삭제 
        del list_word[-1] 
        del list_word[-1]
        if list_word[-1] in neg_vowels: # 음성모음
            list_word[-1] = list_word[-1] + 'ㄹ러'
        elif list_word[-1] in pos_vowels: # 양성모음
            list_word[-1] = list_word[-1] + 'ㄹ라'
        word = "".join(list_word)
        word = join_jamos(word)
        if org_word == '들르': # 들르 예외처리.
            word = '들러'        
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')


    # -U 불규칙
    if org_inf == 'VERB-U' or org_inf == 'ADJ-U': # e.g. 갈겨쓰(양성모음), 가냘프(음성모음)
        LOG(org__info)
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if len(list_word) == 2:
            list_word[-1] = 'ㅓ' # 끄,뜨,크,트는 그냥 모두 음성모음으로 처리
        else:
            del list_word[-1] # '쓰'의 'ㅡ'삭제 
            posneg_vowel = list_word[-3] + list_word[-2] # 이런식으로 segmentation 필요. 항상 [-2]가 모음이 아닐 수도 있다. 
            neg_vowel = list(set(posneg_vowel).intersection(set(neg_vowels)))
            pos_vowel = list(set(posneg_vowel).intersection(set(pos_vowels)))
            if len(neg_vowel) != 0: 
                list_word[-1] = list_word[-1] + 'ㅓ' # 음성모음이면
            else: 
                list_word[-1] = list_word[-1] + 'ㅏ' # 양성모음이면
        word = "".join(list_word)
        word = join_jamos(word)    
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) # 확인
        LOG('\n')


    # -LE 불규칙
    if org_inf == 'VERB-LE' or org_inf == 'ADJ-LE': # (러불규칙) e.g. 이르
        LOG(org__info)
        ### VERB-S1 # e.g. 이르러
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word += '러'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) 
        LOG('\n')


    if org_inf == 'VERB-WU': # e.g. 푸
        LOG(org__info)
        ### VERB-WU1 # e.g. 퍼
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        list_word[-1] = 'ㅓ'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '1'
        # anly
        anly = org_word+'/'+pos
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info)
        LOG(word +' ---> '+ _info) 
        LOG('\n')


    if org_inf == 'ADJ-H': # e.g. 이렇 / 하얗/ 빨갛
        LOG(org__info)
        ### ADJ-H1 # e.g. 이러 / 빨가
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        del list_word[-1] # 마지막 글자인 ㅎ 제거
        if not (list_word[-1] == 'ㅑ' or list_word[-1] == 'ㅕ'):
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '1'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            LOG(word +' ---> '+ _info)  
        ### ADJ-H2 # e.g. 이러 / 빨가
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        del list_word[-1] # 마지막 글자인 ㅎ 제거
        if not (list_word[-1] == 'ㅑ' or list_word[-1] == 'ㅕ'):
            list_word[-1] = 'ㅐ'
        else:
            if list_word[-1] == 'ㅑ':
                list_word[-1] = 'ㅒ'
            else: # 'ㅕ'이면..
                list_word[-1] = 'ㅖ'
        word = "".join(list_word)
        word = join_jamos(word)
        # pos -> 그대로 사용.
        # inf
        inf = org_inf + '2'
        # anly
        anly = org_word+'/'+pos # _info에는 원형단어(org_word)가 들어간다.
        _info = pos+' '+inf+' '+anly
        trie.insert(word, _info) # trie 넣을 때는 변형단어(word)를 넣는다.
        LOG(word +' ---> '+ _info) 
        LOG('\n')


    if org_inf == 'VERB-REG' or org_inf == 'ADJ-REG': # e.g. ㅗ, ㅜ, ㅣ, ㅚ
        ### VERB-REG4 # e.g. 나오 -> 나와 
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if list_word[-1] == 'ㅗ':
            LOG(org__info)
            list_word[-1] = 'ㅘ'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '4'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            LOG(word +' ---> '+ _info) # 확인
            LOG('\n')
        ### VERB-REG4 # e.g. 세우 -> 세워 
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if list_word[-1] == 'ㅜ':
            LOG(org__info)
            list_word[-1] = 'ㅝ'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '4'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            LOG(word +' ---> '+ _info) # 확인
            LOG('\n')
        ### VERB-REG4 # e.g. 옮기 -> 옮겨 
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if list_word[-1] == 'ㅣ':
            LOG(org__info)
            list_word[-1] = 'ㅕ'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '4'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            LOG(word +' ---> '+ _info) # 확인
            LOG('\n')
        ### VERB-REG4 # e.g. 사람되 -> 사람돼 
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if list_word[-1] == 'ㅚ':
            LOG(org__info)
            list_word[-1] = 'ㅙ'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '4'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            LOG(word +' ---> '+ _info) # 확인
            LOG('\n')

            
    if org_inf == 'VERB-REG': #or org_inf == 'ADJ-REG':            
        ### VERB-REG4 # e.g. 사람되 -> 사람돼 
        # word
        word = split_syllables(org_word)
        list_word = list(word)
        if list_word[-2] == 'ㅓ' and list_word[-1] == 'ㄱ':
            print(org__info)
            list_word[-1] = 'ㅙ'
            word = "".join(list_word)
            word = join_jamos(word)
            # pos -> 그대로 사용.
            # inf
            inf = org_inf + '4'
            # anly
            anly = org_word+'/'+pos
            _info = pos+' '+inf+' '+anly
            trie.insert(word, _info)
            print(word +' ---> '+ _info) # 확인
            print('\n')
#print('>> Store 확장-사전 Corpus to Trie, Complete !!')

VV VERB-REG 갉아먹/VV
갉아머ㅙ ---> VV VERB-REG4 갉아먹/VV


VV VERB-REG 겁먹/VV
겁머ㅙ ---> VV VERB-REG4 겁먹/VV


VV VERB-REG 까먹/VV
까머ㅙ ---> VV VERB-REG4 까먹/VV


VV VERB-REG 뜯어먹/VV
뜯어머ㅙ ---> VV VERB-REG4 뜯어먹/VV


VV VERB-REG 마음먹/VV
마음머ㅙ ---> VV VERB-REG4 마음먹/VV


VV VERB-REG 맞먹/VV
맞머ㅙ ---> VV VERB-REG4 맞먹/VV


VV VERB-REG 먹/VV
머ㅙ ---> VV VERB-REG4 먹/VV


VX VERB-REG 먹/VX
머ㅙ ---> VX VERB-REG4 먹/VX


VV VERB-REG 빌어먹/VV
빌어머ㅙ ---> VV VERB-REG4 빌어먹/VV


VV VERB-REG 빨아먹/VV
빨아머ㅙ ---> VV VERB-REG4 빨아먹/VV


VV VERB-REG 빼먹/VV
빼머ㅙ ---> VV VERB-REG4 빼먹/VV


VV VERB-REG 썩/VV
써ㅙ ---> VV VERB-REG4 썩/VV


VV VERB-REG 얻어먹/VV
얻어머ㅙ ---> VV VERB-REG4 얻어먹/VV


VV VERB-REG 잡아먹/VV
잡아머ㅙ ---> VV VERB-REG4 잡아먹/VV


VV VERB-REG 적/VV
저ㅙ ---> VV VERB-REG4 적/VV


VV VERB-REG 처먹/VV
처머ㅙ ---> VV VERB-REG4 처먹/VV


VV VERB-REG 퍼먹/VV
퍼머ㅙ ---> VV VERB-REG4 퍼먹/VV


VV VERB-REG 해먹/VV
해머ㅙ ---> VV VERB-REG4 해먹/VV




In [6]:
### 확장사전 entry 예시
# VV VERB-L 가득들/VV
# 가득드 ---> VV VERB-L1 가득들/VV


# VV VERB-REG 가라앉히/VV
# 가라앉혀 ---> VV VERB-REG4 가라앉히/VV


# VA ADJ-P 가렵/VA
# 가려우 ---> VA ADJ-P1 가렵/VA
# 가려워 ---> VA ADJ-P2 가렵/VA


# VV VERB-REG 가로놓이/VV
# 가로놓여 ---> VV VERB-REG4 가로놓이/VV


# VV VERB-S 가로젓/VV
# 가로저 ---> VV VERB-S1 가로젓/VV


# VV VERB-LU 가로지르/VV
# 가로질러 ---> VV VERB-LU1 가로지르/VV

# 3. 신조어 추가
* 보통 신조어는 '용언'에 해당되므로 원형 그대로 사전에 등록한다.
* 신조어가 등장할 경우 위와 똑같은 포멧으로 추가해준다. ex. 신조어 NN NounC 신조어/NN
* Inflection 추가시에 NounC(을), NounV(를) 구분할 것.

# 4. 형태소 분석 알고리즘
* 입력을 모두 cover하는 분석 결과 찾기
    - segment를 어떻게 할 것인가 -> '최장일치' 기준
    - pos ambiguity는 어떻게 해결할 것인가 -> '접속정보' 또는 'POS Tagger'를 사용

![](resources/pic.png)
* 위의 그림과 같이 초성, 중성, 종성으로 나눠야 하지만, 우리는 나누지 않았다. 그래서 '중'같은 entry를 찾게 되지만 어차피 조합하는 과정에서 시, 저, 고저와 함께 '종'과 같이 이어지지 않는 독립적인 entry들은 삭제된다.

### 입력 모양 
* 가정 1: 공백이 없는 하나의 어절을 가진다고 가정한다.
* 가정 2: 문법적 오류가 없다고 가정한다. 
* 가정 3: 기능어가 존재할 경우 맨 뒷부분에 단 '1개'의 기능어가 있다고 가정한다. (단, ㄴ, ㄹ, ㄻ, ㅁ, ㅆ 기능어는 제외)
<br>
( 가정 3으로 형태소 분석 결과 경우의 수를 줄일 수 있다. 예를 들어 '신'에서 '시', '신'을 뽑을 수 있는데, 기능어가 'ㄴ'이 있으므로 '시'라는 단어가 조합을 가지게 된다. ) 


In [7]:
target_word = '알려졌다'
target_word = '태양이'
#target_word = '한다든가'
#target_word = '가'
#target_word = '급가증스러웠다고'
#target_word = '있었다'


""" 1 ~ 6 : Segmenting using 최장일치 """
""" 7 ~ : POS Ambiguity 해결 using 접속정보 """

########################################
""" 1. 모든 경우의 entry-analysis 저장 """
########################################
# 모음,자음단위로 나눠서(decomposition하여) 모든 경우의 조합들을 사전 lookup을 통해 뽑아낸다.
### decomposition
target_word = split_syllables(target_word)
len_target_word = len(target_word)
### lookup all entries
# 원래 엔트리가 초성, 중성, 종성이 모두 있는데 초성, 중성까지만 있는 entry는 삭제 
# 즉, 해당 엔트리 바로 오른쪽에 자음이 두 번 등장하는 경우 삭제 
# (e.g. '신고전주의성진을'를 형태소 분석할 때, '시', '고저' 엔트리들은 삭제, 고려해볼 필요도 없다.)
collect_entry_anly = [] # 모든 경우의 entry 정보가 들어감.
#print(len_target_word)

""" 순방향 스캔 for 엔트리-사전 """
for i, _ in enumerate(target_word):
    for j, _ in enumerate(target_word[i:]):
        #print(target_word[i:][:j+1], i, j+i)
        if not search(trie, target_word[i:][:j+1]) == 'No such word in Trie':
            entry = join_jamos(target_word[i:][:j+1])
            anly = search(trie, target_word[i:][:j+1])
            collect_entry_anly.append([(entry)]+[i]+[j+i]+['ENT']+anly)     

""" 역방향 스캔 for 기능어-사전 """
toggle_fun = False
#one_ch_fun = ['ㄴ','ㄹ','ㄻ','ㅁ','ㅆ']
target_word = target_word[::-1]
for i, _ in enumerate(target_word):
    for j, _ in enumerate(target_word[i:]):
        #print(target_word[i:][:j+1], len_target_word-1-i, len_target_word-1-j-i)
        if not search(trie_fun, target_word[i:][:j+1]) == 'No such word in Trie':

            #if i==0: # 기능어는 한 개만 (뒤에서부터 검색) 찾아서 넣어준다. 하지만, 기존 기능어를 포함하는 더 큰 범위의 기능어가 있으면 또 추가한다. (즉, i=0이면..) 단, 포함하지 않으면 추가하지 않는다.
            entry = join_jamos(target_word[i:][:j+1][::-1])
            anly = search(trie_fun, target_word[i:][:j+1])
            collect_entry_anly.append([(entry)]+[len_target_word-1-j-i]+[len_target_word-1-i]+['FUN']+anly) 

#             if toggle_fun == False
#                 if entry not in one_ch_fun: # 하나의 자음으로 이뤄진 기능어는 count하지 않는다. '을' 기능어에 'ㄹ'기능어가 또 있다. 이것은 나중에 다시 처리한다.
#                     toggle_fun = True

# 2개의 trie를 사용했지만, 하나의 dict에 저장한다.

In [8]:
collect_entry_anly

[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
 ['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
 ['야', 2, 3, 'ENT', 'NNG NounV 야/NNG'],
 ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
 ['이',
  5,
  6,
  'ENT',
  'NP NounV 이/NP',
  'XPN NounV 이/XPN',
  'XSN NounV 이/XSN',
  'VV VERB-L1 일/VV',
  'VV VERB-S1 잇/VV'],
 ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS'],
 ['야',
  2,
  3,
  'FUN',
  'E A-13-0 야/E',
  'E VA-5-0 야/E',
  'J N-10-0 야/JKQ',
  'J N-2-0 야/JKQ',
  'P C-5-2 이/VCP+야/E']]

In [9]:
#################################################################
""" 2. 전처리 (entry와 index모두 똑같은 entry-analysis 서로 병합) """
#################################################################
# 예를 들어, 엔트리-사전의 '신'과 기능어-사전의 '신'이 index range마저 똑같다면, 하나의 '신'으로 합쳐준다.
# 그런데, 맨 뒷부분에서 기능어는 1개만 가지기 때문에 엔트리-사전과 겹칠일이 많치는 않다.
# 예를 들어, 을 같은 경우는 엔트리-사전과 기능어-사전에 동시에 있고 index도 같을 수 있다. (하지만 여기서 엔트리-사전에 '을'이 없다.)
# 윗 단에서 최대한 경우의 수를 줄여줘야 한다. 그렇지 않으면, 밑으로 갈 수록 경우의 수는 기하급수적으로 늘어날 수 있다.
# 단, 병합할 때 같은 종류의 엔트리만 고려한다. 즉, FUN, ENT 사이의 병합은 하지 않는다. EX. '을' 같은 경우는 2가지 모두 존재한다.
for i, _ in enumerate(collect_entry_anly):
    for j in range(0, len(collect_entry_anly)):
        if not (j == i or collect_entry_anly[j][3] != collect_entry_anly[i][3]): 
            # 자기 자신과 똑같은 친구는 빼고. 똑같은 ENT or FUN만 고려한다.
            if collect_entry_anly[i][0] == collect_entry_anly[j][0] and collect_entry_anly[i][1] == collect_entry_anly[j][1]:
                collect_entry_anly[i] += collect_entry_anly[j][4:]
                collect_entry_anly[j].insert(0, 'to-be-deleted')   

# remove elements, which has 'to-be-deleted' in the first position
collect_entry_anly = [elem for elem in collect_entry_anly if elem[0] != 'to-be-deleted']

# ###############################################
# """ 3. Longest 기능어만 살리기 (경우의 수 줄이기)"""
# ###############################################
# # 만약 기능어들이 같은 index range를 공유한다면, 가장 긴 기능어를 살리고 나머진 삭제한다. (우리는 기능어가 1개만 있다고 가정을 하였다.)
# # 단, 엔트리들은 이런식으로 삭제하면 안된다. 모두 살려두어 나중에 분석결과에 모두 표시해준다.
# # 예를 들어 'ㄹ' 기능어 (20, 20), '을' 기능어 (18, 20)가 있는 경우 'ㄹ' 기능어를 삭제한다.
# filtered_fun = [i for i in collect_entry_anly if i[3] == 'FUN'] # filtering only fun
# filt_sorted_fun = sorted(filtered_fun, key = lambda x: len(split_syllables(x[0])))
# for i, _ in enumerate(filt_sorted_fun):
#     target = filt_sorted_fun[i]
#     for j in range(i+1, len(filt_sorted_fun)):
#         ref = filt_sorted_fun[j]
#         if int(ref[1]) <= int(target[1]):# and int(ref[2]) >= int(target[2]):
#             filt_sorted_fun[i].insert(len(filt_sorted_fun[i]), 'to-be-deleted') # add on last position

# # remove elements, which has 'to-be-deleted' in the last position
# filt_sorted_fun = [elem for elem in filt_sorted_fun if elem[-1] != 'to-be-deleted']            

# # merge
# filtered_ent = [i for i in collect_entry_anly if i[3] == 'ENT'] # filtering only fun
# final_collect_entry_anly = filtered_ent + filt_sorted_fun

final_collect_entry_anly = collect_entry_anly[:]
###################################################################################
""" 4. 엔트리 중 내 바로 오른쪽 index가 비어있는 경우 해당 엔트리 지우기 (경우의 수 줄이기)"""
###################################################################################
# 예를 들어, 신고전주의정신을 -> 시, 저, 중 은 삭제된다. 다음 index가 비어있기 때문에
for i, _ in enumerate(final_collect_entry_anly):
    tnk = False
    for j in range(0, len(final_collect_entry_anly)):
        if final_collect_entry_anly[i][2]+1 ==  final_collect_entry_anly[j][1]:
            tnk = True
        elif final_collect_entry_anly[i][2]+1 == len_target_word:
            tnk = True
    if tnk == False:
        final_collect_entry_anly[i].insert(0, 'to-be-deleted')

# remove elements, which has 'to-be-deleted' in the first position        
final_collect_entry_anly = [elem for elem in final_collect_entry_anly if elem[0] != 'to-be-deleted']

In [10]:
final_collect_entry_anly

[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
 ['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
 ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
 ['이',
  5,
  6,
  'ENT',
  'NP NounV 이/NP',
  'XPN NounV 이/XPN',
  'XSN NounV 이/XSN',
  'VV VERB-L1 일/VV',
  'VV VERB-S1 잇/VV'],
 ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']]

In [11]:
#########################################################################
""" 5. 조합하기 (엔트리 묶음으로 즉, 엔트리 종류는 1개라고 가정한다 여기서는..) """
#########################################################################
# 조합하는 과정에서, 띄어쓰기 오류 또는 신조어 오류를 확인할 수 있다.
# 오류확인 1. sorting 후에 마지막 element의 start index가 target_word의 길이와 맞지 않을 경우.

def recursive_merging(collection, index_set, path, list_global_cases):
    temp_path = path[:]
    for index in index_set:
        path.append(index)
        index_set = [e for e in collection if int(e[1]) == index[2]+1] # first position
        if len(index_set) == 0: # the end point
            # save to global and then game over
            list_global_cases.append(path)
        else:
            # recursive keep going
            recursive_merging(collection, index_set, path, list_global_cases) # recursive function
        path = temp_path[:] # recursive돌아서 망가진 path 원래대로 복귀

        
# sorting based on start index
final_collect_entry_anly = sorted(final_collect_entry_anly, key = lambda x: int(x[1]))


# 오류확인 1. 처음과 끝의 경계는 아래와 같이 체크하면 된다. (중간에 있는 오류는 다음단계에서 확인한다.)
assert(final_collect_entry_anly[0][1] == 0) # 처음이 index 0으로 시작하지 않을 경우.
assert(final_collect_entry_anly[-1][2] == len_target_word-1) # 마지막이 word길이만큼의 index가 아닐 경우.


### merging / combining all cases using recursive function
list_global_cases, path = [], []
# filtering only index0 (Starting point to recursive function)
index0_entry_anly_set = [ele for ele in final_collect_entry_anly if int(ele[1]) == 0] # first position
# recursive merging
recursive_merging(final_collect_entry_anly, index0_entry_anly_set, path, list_global_cases)

# 오류확인 2. 처음부터 끝까지 이어지는 하나의 case도 없을 경우.
fullpath_check = False
for i in range(0, len(list_global_cases)):
    if list_global_cases[i][-1][2]+1 == len_target_word:
        fullpath_check = True
assert(fullpath_check == True) # 오류 난다면 사전 lookup 문제이다


### Sequence가 0에서부터 len_target_word-1까지 이어지지 않으면 삭제한다.
for i, seq in enumerate(list_global_cases):
    if seq[0][1] != 0 or seq[-1][2] != len_target_word-1:
        list_global_cases[i] = 'to-be-deleted'
# remove elements, which has 'to-be-deleted' in the first position        
list_global_cases = [elem for elem in list_global_cases if elem != 'to-be-deleted']

In [12]:
list_global_cases

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']]]

In [13]:
###########################################################
""" 6. 형태소 조합에서 기능어 있을 때, 마지막에 없는 경우 삭제 """
###########################################################
# FUN이 2번 연속 등장하는 경우 삭제

for i, case in enumerate(list_global_cases):
    case_len = len(case)
    for j, entry in enumerate(case):
        if case[j][3] == 'FUN':
            if j!=case_len-1: # FUN이 마지막에 있지 않은 경우...
                #print(entry)
                list_global_cases[i] = 'to-be-deleted'
# filtering
list_global_cases = [case for case in list_global_cases if case != 'to-be-deleted']

In [14]:
list_global_cases

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']]]

In [15]:
# ##################################################
# """ 6. 최장길이 우선으로 정렬하고 상위 1개만 보이자. """
# ##################################################
# # 위의 print 내용을 보다시피 경우의 수가 많다. 더군다나 entry 종류까지 unfolding하면 경우의 수가 매우 많이 증가한다.
# # 따라서, '최장일치' 우선순위 조건을 부여해, 길이가 긴 순서대로 정렬한 후 상위 1개까지만 확인하자.
# temp_summed_squared = [0] * len(list_global_cases)
# for i, case in enumerate(list_global_cases):
#     # 최장일치 기준: 모든 엔티티 길이의 제곱들을 각각 더한 값이 제일 큰 case가 최장일치가 된다.
#     # 모든 엔티티길이를 고려한 이유: maximum길이를 가지는 엔티티만 고려하면 중복인 경우가 생긴다.
#     # 제곱을 한 이유: maximum길이를 가지는 엔티티는 똑같을 때, 다른 엔티티의 길이가 1인 2개와, 2인 1개가 있을 경우, 2에 더 가중치를 주기 위해 제곱한다.
#     for j, entity in enumerate(case):
#         temp_summed_squared[i] += (len(split_syllables(case[j][0])) * len(split_syllables(case[j][0]))) # 제곱

# max_value = max(temp_summed_squared)
# max_info_cases = []

# for i, value in enumerate(temp_summed_squared):
#     if value == max_value:
#         max_info_cases.append(list_global_cases[i])

# ### 의문점: 최장길이 우선으로 상위 1개를 선택하는데, 이런 경우 접속정보를 사용하지 않아도 항상 정상적인 연결이 되는가?

max_info_cases = list_global_cases[:]

In [16]:
# output이 1개의 case가 아닌 여러개의 case들도 생길 수 있다.
# 여러개의 case가 생길경우 하나로 줄일 수 있는 방법은 pos tagger를 사용하는 방법밖에 없다.
max_info_cases

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']]]

In [17]:
len(max_info_cases)

4

In [18]:
# 접속정보 LOAD하기
wb_connect_info = xlrd.open_workbook('resources/connect_info.xlsx') 
ws_entent = wb_connect_info.sheet_by_name('내용어끼리')
num_rows = ws_entent.nrows #줄 수 가져오기
num_cols = ws_entent.ncols #칸 수 가져오기

In [19]:
entent_lookup = []
for i in range(1, num_rows):
    entent_lookup.append(ws_entent.cell_value(i, 0)+' '+ws_entent.cell_value(i, 1))
    #print(ws_entent.cell_value(i, 0), ws_entent.cell_value(i, 1))
#     word = worksheet1.cell_value(i, 0)
#     pos = worksheet1.cell_value(i, 1)
#     inf = worksheet1.cell_value(i, 2)
#     anly = word+'/'+pos
#     _info = pos+' '+inf+' '+anly
#     trie.insert(word, _info) # word는 insert함수에 의해 decomposition된다.
# print('>> Store 엔트리-사전 Corpus to Trie, Complete !!')  

In [20]:
#wb_connect_info = xlrd.open_workbook('connect_info.xlsx') 
ws_entfun = wb_connect_info.sheet_by_name('내용어와기능어')
num_rows = ws_entfun.nrows #줄 수 가져오기
num_cols = ws_entfun.ncols #칸 수 가져오기

In [21]:
entfun_lookup = []
for i in range(1, num_rows):
    entfun_lookup.append(ws_entfun.cell_value(i, 0)+' '+ws_entfun.cell_value(i, 1)+' '+ws_entfun.cell_value(i, 2))
    #print(ws_entent.cell_value(i, 0), ws_entent.cell_value(i, 1))
#     word = worksheet1.cell_value(i, 0)
#     pos = worksheet1.cell_value(i, 1)
#     inf = worksheet1.cell_value(i, 2)
#     anly = word+'/'+pos
#     _info = pos+' '+inf+' '+anly
#     trie.insert(word, _info) # word는 insert함수에 의해 decomposition된다.
# print('>> Store 엔트리-사전 Corpus to Trie, Complete !!')  

In [22]:
# 원형 전처리 (ex. VERB-S0 -> VERB-S) 위의 불규칙 사전 등록시에 원형에 0번호는 입력하지 않아서 여기서 빼줘야 한다.
for i, entry in enumerate(entfun_lookup):
    if list(entry.split()[0])[-1] == '0':
        temp_list = list(entry.split()[0])
        del temp_list[-1]
        temp_list = "".join(temp_list)
        entfun_lookup[i] = temp_list+' '+entry.split()[1]+' '+entry.split()[2]

In [23]:
max_info_cases

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']]]

In [24]:
##############################################################
""" 7. 각각의 엔티티의 pos의 경우의 수를 줄이자. (접속 정보 사용) """
##############################################################
# 만약에 case들이 단 하나의 entry로 구성되어있다면 이 단계는 필요없다.
# 이제 하나의 case을 얻었으니, 각각의 엔티티의 pos의 경우의 수를 줄이자. 보통 하나의 엔티티는 여러개 종류의 pos를 가진다. 이것을 그대로 펼치면 경우의 수가 또 많아진다.
# segmenting을 하기전에 pos ambiguity를 해결해도 된다. 하지만, 계산량이 엄청날 것이다.

### 공지
# entent lookup table (내용어내용어)은 사용하지 말자.
# 일단 정확한 접속 규칙이 아니고 빈도수로 나와 있다. 이것은 접속 여부를 결정해주는 것은 아니다.


list_delete_idx = []

for i, case in enumerate(max_info_cases):    
    if len(max_info_cases[i]) != 1: # 단 하나의 entry로 구성되어있다면 이 단계는 필요없다. 이웃 entry가 아예 없기 때문이다.
    
        ### FUN이 있는 경우랑 없는 경우랑 나눈다. 있으면 FUN Scanning을 해야되니까..
        filtered_fun = [i for i in max_info_cases[i] if i[3] == 'FUN']
        
#         # FUN이 한개도 없는 경우
#         if len(filtered_fun) == 0:
            
#             # RIGHT-scanning
#             for j, entry in enumerate(max_info_cases[i]): 
#                 if not j==len(max_info_cases[i])-1: # except for last one
#                     entry_list_t0 = max_info_cases[i][j][4:]
#                     entry_list_t1 = max_info_cases[i][j+1][4:]
                    
#                     for m, anly_info_t0 in enumerate(entry_list_t0):
#                         tkn = False
#                         pos_t0 = max_info_cases[i][j][4:][m].split()[0]
#                         #print(pos)
#                         for n, anly_info_t1 in enumerate(entry_list_t1):
#                             pos_t1 = max_info_cases[i][j+1][4:][n].split()[0]
#                             temp = pos_t0+' '+pos_t1
                            
#                             if temp in entent_lookup:
#                                 tkn = True
                        
#                         if tkn == False and len(entry_list_t0)!=1:
# #                             print(len(entry_list_t0))
# #                             print(max_info_cases[i][j][4:][m]) 
# #                             print('->', pos_t0, pos_t1)
#                             tuple_ = (i, j, m)
#                             list_delete_idx.append(tuple_)
#                             #print(temp)                            

                            
 
#             # LEFT-scanning        
#             for j, entry in reversed(list(enumerate(max_info_cases[i]))):
#                 #print(j, entry)
#                 if not j==0: # except for first one
#                     entry_list_t0 = max_info_cases[i][j][4:]
#                     entry_list_tmin1 = max_info_cases[i][j-1][4:]
#                     #print(entry_list_t0)
                    
#                     for m, anly_info_t0 in enumerate(entry_list_t0):
#                         tkn2 = False
#                         pos_t0 = max_info_cases[i][j][4:][m].split()[0]
#                         #print(pos)
#                         for n, anly_info_tmin1 in enumerate(entry_list_tmin1):
#                             pos_tmin1 = max_info_cases[i][j-1][4:][n].split()[0]
#                             temp = pos_tmin1+' '+pos_t0
                            
#                             if temp in entent_lookup:
#                                 tkn2 = True
                            
#                         if tkn2 == False and len(entry_list_t0)!=1:
# #                             print(len(entry_list_t0))
# #                             print(max_info_cases[i][j][4:][m])
# #                             print('<-', pos_tmin1, pos_t0)
#                             tuple_ = (i, j, m)
#                             list_delete_idx.append(tuple_)
#                             #print(temp)                           
                        
              
      
        
        # FUN이 한개라도 있는 경우
        if len(filtered_fun) != 0:
            
#             # RIGHT-scanning (마지막 1개는 scan하지 않는다.) 
#             for j, entry in enumerate(max_info_cases[i]): 
#                 if not (j==len(max_info_cases[i])-2 or j==len(max_info_cases[i])-1): # except for last one
#                     entry_list_t0 = max_info_cases[i][j][4:]
#                     entry_list_t1 = max_info_cases[i][j+1][4:]
                    
#                     for m, anly_info_t0 in enumerate(entry_list_t0):
#                         tkn = False
#                         pos_t0 = max_info_cases[i][j][4:][m].split()[0]
#                         #print(pos)
#                         for n, anly_info_t1 in enumerate(entry_list_t1):
#                             pos_t1 = max_info_cases[i][j+1][4:][n].split()[0]
#                             temp = pos_t0+' '+pos_t1
                            
#                             if temp in entent_lookup:
#                                 tkn = True
                        
#                         if tkn == False and len(entry_list_t0)!=1:
# #                             print(len(entry_list_t0))
# #                             print(max_info_cases[i][j][4:][m]) 
# #                             print('->', pos_t0, pos_t1)
#                             tuple_ = (i, j, m)
#                             list_delete_idx.append(tuple_)
#                             #print('RIGHT===',temp, tuple_)              
            
            
#             # LEFT-scanning (마지막 2번째부터 시작한다.) 
#             for j, entry in reversed(list(enumerate(max_info_cases[i]))):
#                 #print(j, entry)
#                 if not (j==0 or j==len(max_info_cases[i])-1):
#                     entry_list_t0 = max_info_cases[i][j][4:]
#                     entry_list_tmin1 = max_info_cases[i][j-1][4:]
#                     #print(entry_list_t0)
                    
#                     for m, anly_info_t0 in enumerate(entry_list_t0):
#                         tkn2 = False
#                         pos_t0 = max_info_cases[i][j][4:][m].split()[0]
#                         #print(pos)
#                         for n, anly_info_tmin1 in enumerate(entry_list_tmin1):
#                             pos_tmin1 = max_info_cases[i][j-1][4:][n].split()[0]
#                             temp = pos_tmin1+' '+pos_t0
                            
#                             if temp in entent_lookup:
#                                 tkn2 = True
                            
#                         if tkn2 == False and len(entry_list_t0)!=1:
# #                             print(len(entry_list_t0))
# #                             print(max_info_cases[i][j][4:][m])
# #                             print('<-', pos_tmin1, pos_t0)
#                             tuple_ = (i, j, m)
#                             list_delete_idx.append(tuple_)
#                             #print('LEFT===',temp, tuple_)            
            
            
            
            
            ############################################################################# FUN처리
            # RIGHT-scanning
            for j, entry in enumerate(max_info_cases[i]):
                if j==len(max_info_cases[i])-2:
                    
                    entry_list_ref = max_info_cases[i][j][4:]
                    entry_list_comp = max_info_cases[i][j+1][4:]
                    #print(entry_list_t0)
                    
                    for m, anly_info_ref in enumerate(entry_list_ref): # 기준
                        tkn3 = False
                        pos_ref = max_info_cases[i][j][4:][m].split()[0]
                        inf_ref = max_info_cases[i][j][4:][m].split()[1]
                        #print(pos)
                        for n, anly_info_comp in enumerate(entry_list_comp):
                            pos_comp = max_info_cases[i][j+1][4:][n].split()[0]
                            inf_comp = max_info_cases[i][j+1][4:][n].split()[1]
                            temp = inf_ref+' '+pos_comp+' '+inf_comp
                            #print(temp, '[right]')
                            if temp in entfun_lookup:
                                tkn3 = True
                            
                        if tkn3 == False and len(entry_list_ref)!=1:
#                             print(len(entry_list_t0))
#                             print(max_info_cases[i][j][4:][m])
#                             print('<-', pos_tmin1, pos_t0)
                            tuple_ = (i, j, m)
                            list_delete_idx.append(tuple_)
                            #print('[FUN] right------------>', temp, tuple_)                     
                
                
            # LEFT-scanning
            for j, entry in reversed(list(enumerate(max_info_cases[i]))):
                #print(j, entry)
                if j==len(max_info_cases[i])-1: 
                    entry_list_ref = max_info_cases[i][j][4:]
                    entry_list_comp = max_info_cases[i][j-1][4:]
                    #print(entry_list_t0)
                    
                    for m, _ in enumerate(entry_list_ref): # 기준
                        tkn4 = False
                        ref_pos = max_info_cases[i][j][4:][m].split()[0]
                        ref_inf = max_info_cases[i][j][4:][m].split()[1]
                        #print(pos)
                        for n, _ in enumerate(entry_list_comp):
                            comp_inf = max_info_cases[i][j-1][4:][n].split()[1]
                            temp = comp_inf+' '+ref_pos+' '+ref_inf # LEFT-scanning이기 때문에...
                            #print(temp, '[left]')
                            if temp in entfun_lookup:
                                tkn4 = True
                            
                        if tkn4 == False and len(entry_list_ref)!=1:
#                             print(len(entry_list_t0))
#                             print(max_info_cases[i][j][4:][m])
#                             print('<-', pos_tmin1, pos_t0)
                            tuple_ = (i, j, m)
                            list_delete_idx.append(tuple_)
                            #print('[FUN] left------------>', temp, tuple_)   


                

In [25]:
len(max_info_cases)

4

In [26]:
for j, entry in reversed(list(enumerate(max_info_cases[i]))):
    print(j, entry)

1 ['이', 5, 6, 'FUN', 'E A-1-0 이/EF', 'J N-11-0 이/JKS']
0 ['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG']


In [27]:
list_delete_idx = list(set(list_delete_idx))[:] # A set is guaranteed to not have duplicates.
list_delete_idx = sorted(list_delete_idx, key=lambda x: (int(str(x[0])+str(x[1])+str(x[2]))), reverse=True)

In [28]:
list_delete_idx

[(3, 1, 0), (1, 2, 0)]

In [29]:
for index_set in list_delete_idx:
    i = index_set[0]
    j = index_set[1]
    m = index_set[2]
    #print(i, j, m)
    
    #print(max_info_cases[i][j][4:])
    #print(max_info_cases[i][j][4:][m])
    filtered = [elem for elem in max_info_cases[i][j][4:] if elem != max_info_cases[i][j][4:][m]]
    #print(filtered)
#     print(max_info_cases[i][j])
#     print(max_info_cases[i][j][0:4] + filtered)
#     print('\n')
    
    max_info_cases[i][j] = max_info_cases[i][j][0:4] + filtered
    #max_info_cases[i][j] = max_info_cases[i][j][0:3]+filtered

In [30]:
max_info_cases

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'J N-11-0 이/JKS']]]

In [31]:
##############################################################
""" 8. 하나의 엔트리라도 형태소 추출 실패한 경우 해당 case 삭제하기 """
##############################################################        
fail = False
failure_check = [0] * len(max_info_cases)
for i, case in enumerate(max_info_cases):
    for j, entry in enumerate(case):
        if len(entry)==4:
            fail = True
    if fail == True:
        failure_check[i] = 1

    fail = False
    
list_finalresult = []
for i, case in enumerate(max_info_cases):
    if failure_check[i] == 0:
        list_finalresult.append(case)

In [32]:
list_finalresult

[[['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태', 0, 1, 'ENT', 'NNG NounV 태/NNG'],
  ['양', 2, 4, 'ENT', 'NNG NounC 양/NNG', 'XPN NounC 양/XPN', 'XSN NounC 양/XSN'],
  ['이', 5, 6, 'FUN', 'J N-11-0 이/JKS']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이',
   5,
   6,
   'ENT',
   'NP NounV 이/NP',
   'XPN NounV 이/XPN',
   'XSN NounV 이/XSN',
   'VV VERB-L1 일/VV',
   'VV VERB-S1 잇/VV']],
 [['태양', 0, 4, 'ENT', 'NNG NounC 태양/NNG'],
  ['이', 5, 6, 'FUN', 'J N-11-0 이/JKS']]]

### 출력되는 경우의 수에 대하여...


형태소 분석의 최소한의 역할은 morphological(형태학의) 오류가 없는지 확인하는 것이다. 엔트리-기능어 사전을 통해 나오는 경우의 수에서 형태학의 오류가 있는 경우는 제거를 한다. 다시 말해, 형태소분석은 **접속정보**만 보고 해당되지 않는 것들은 제거한다. 이렇게 최소한의 역할을 하는 형태소 분석의 결과값은 경우의 수가 매우 많아진다. 엔트리별로 경우의 수가 있고, 각 엔트리에서 POS별로 경우의 수가 있어서 기하급수적으로 늘어난다. 따라서, 경우의 수를 최대한 줄여주기 위해 '최장일치' 등의 테크닉으로 경우의 수를 많이 줄여줄 수 있다. 

하지만 궁극적으로 경우의 수를 줄여 최적의 경우를 찾아내는 것은 POS Tagger가 할 일이다. 물론, 섬세하고 heuristic기법이 많이 반영된 형태소 분석기가 최적의 경우를 찾아내기도 하다.

In [33]:
# for entry_analysis in (collect_entry_anly):
#     if entry_analysis[0] == 'to-be-deleted':
#         collect_entry_anly.remove(entry_analysis)
# i = 0
# while i < len(collect_entry_anly):
#     if collect_entry_anly[i][0] == 'to-be-deleted':
#         del collect_entry_anly[i]
#     else:
#         i+=1